In [ ]:
import json
import math
import os
import sys
import typing as T

import dotenv
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)

SRC_DIR = os.path.join(ROOT_DIR, 'src')
sys.path.append(SRC_DIR)

DATA_DIR = os.path.join(CURRENT_DIR, "data")

dotenv.load_dotenv()

API_KEY = os.getenv("GOOGLE_MAPS_PLACES_API_KEY")

assert API_KEY, "API_KEY not found in environment variables"

COST_PER_SEARCH = 0.04
MAX_COST_PER_CITY = 700.0

TGTG_DATA_CSV = os.path.join(DATA_DIR, "tgtg_data.csv")

In [ ]:
from search_context.google_places import (
    GooglePlacesAPI,
    ADVANCED_FIELDS,
    ADVANCED_PROMPT,
)
from search_context.util import (
    calculate_cost_from_results,
    extract_city,
    get_city_center_coordinates,
    get_search_grid_details,
    get_viewport,
)
from search_context.util import METERS_PER_MILE, METERS_PER_KILOMETER, DEFAULT_PROMPTS

px.set_mapbox_access_token(os.getenv("MAPBOX_API_KEY"))
google_places = GooglePlacesAPI(API_KEY, verbose=False)

In [ ]:
LOCATIONS = {
    "Portland": {
        "latitude": 45.5152,
        "longitude": -122.676483,
    },
    "New York": {
        "latitude": 40.71,
        "longitude": -74.01,
    },
}

In [ ]:
def save_json(filename, prompt: str, fields: T.List[str], data: T.Dict[str, T.Any]):
    if "places" not in data:
        print("No places found")
        return

    print(f"Number of results saved to file: {len(data['places'])}")

    filename = os.path.join(DATA_DIR, filename)

    print(f"Saving to {filename}")

    with open(filename, "w") as f:
        data = {"prompt": prompt, "fields": fields, "results": results}
        json.dump(data, f, indent=2)

In [ ]:
radius_miles = 20.0
radius_meters = radius_miles * METERS_PER_MILE
scale_factor = 0.5

# Load the data from the CSV
df = pd.read_csv(TGTG_DATA_CSV)

# Create a summary of the stores
store_summary = (
    df.groupby("store_name")
    .agg(
        count=("store_name", "size"),
        latitude=("pickup_location:location:latitude", "first"),
        longitude=("pickup_location:location:longitude", "first"),
    )
    .reset_index()
)
# Add a marker size based on the count and a normalized count for color scaling
store_summary["marker_size"] = store_summary["count"].apply(
    lambda count: max(math.sqrt(count) * scale_factor, 1)
)
store_summary["normalized_count"] = store_summary["count"] / store_summary["count"].max()


# Get the first store's location and dynamically determine the city we are
# working with and the city center coordinates
city = None
store_index = 0
while not city and store_index < len(store_summary):
    first_store_latitude = store_summary["latitude"].iloc[store_index]
    first_store_longitude = store_summary["longitude"].iloc[store_index]

    geolocator = Nominatim(user_agent="tgtg")
    location = geolocator.reverse((first_store_latitude, first_store_longitude), exactly_one=True)
    city = extract_city(location.address)
    store_index += 1

print(f"Latitude: {first_store_latitude}")
print(f"Longitude: {first_store_longitude}")
print(f"City: {city}")

In [ ]:
city = "New York City"

city_center_coordinates = get_city_center_coordinates(city)
assert city_center_coordinates, f"Location not found for {city}"

center_lat, center_lon = city_center_coordinates

# Get the maximum width of the viewport for our search to have good resolution
# since places api limits the search results to 20 max regardless of the radius
print(f"City center: {city_center_coordinates}")
print("Finding maximum viewpoint width")
max_grid_resolution_width_meters = google_places.find_maximum_viewpoint_width(
    center_lat, center_lon, DEFAULT_PROMPTS[0]
)

grid, city_center_coordinates, num_grid_squares, total_cost, new_radius_meters = (
    get_search_grid_details(
        city,
        max_grid_resolution_width_meters,
        radius_meters,
        MAX_COST_PER_CITY,
        COST_PER_SEARCH,
    )
)

print(f"Number of grid squares: {num_grid_squares}")
print(f"Actual radius: {new_radius_meters / METERS_PER_MILE:.2f} miles")
print(f"Actual search blocks: {len(grid)}")
print(f"Total cost: {total_cost:.2f}")

grid_centers = [(x["center"]["latitude"], x["center"]["longitude"]) for x in grid]
grid_df = pd.DataFrame(grid_centers, columns=["latitude", "longitude"])

# Create a map showing the grid and the stores to visually inspect the coverage
fig = go.Figure(
    go.Scattermapbox(
        lat=grid_df["latitude"],
        lon=grid_df["longitude"],
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=2,
            color="blue",
        ),
    )
)

fig.add_trace(
    go.Scattermapbox(
        lat=store_summary["latitude"],
        lon=store_summary["longitude"],
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=store_summary["marker_size"],
            color=store_summary["normalized_count"],
            colorscale="viridis",
            showscale=True,
            cmin=0,
            cmax=1,
        ),
        text=store_summary["store_name"],  # Optionally, set hover text to store name
    )
)

subtext = (
    f"{radius_miles:.1f}mi radius, "
    f"{max_grid_resolution_width_meters}m blocks, {num_grid_squares} blocks"
)
fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.getenv("MAPBOX_API_KEY"),
        bearing=0,
        center=go.layout.mapbox.Center(lat=center_lat, lon=center_lon),
        pitch=0,
        zoom=9,
    ),
    height=800,
    width=1000,
    title_text=f"Search Grid of {city} [~${total_cost:.2f}]",
    title_x=0.5,
    annotations=[
        dict(
            text=subtext,
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.98,
            xanchor="center",
            yanchor="top",
            font=dict(size=14, color="red"),
        )
    ],
)

fig.show()

In [ ]:
prompt = DEFAULT_PROMPTS[0]
fields = ADVANCED_FIELDS
radius_miles = 0.5
# radius_meters = radius_miles * METERS_PER_MILE
radius_meters = 350.0
location = "New York"

latitude = LOCATIONS[location]["latitude"]
longitude = LOCATIONS[location]["longitude"]

rect_viewpoint = get_viewport(latitude, longitude, radius_meters)

data = {"locationRestriction": {"rectangle": rect_viewpoint}}
results = google_places.text_search(prompt, fields, data)
if "places" not in results:
    print("No results found")
else:
    filename = f"{latitude}_{longitude}_{radius_miles}_location_rect_viewpoint_search.json"
    save_json(filename, prompt, fields, results)

In [ ]:
radius_miles = 0.5
latitude = 45.54
longitude = -122.98
prompt = ADVANCED_PROMPT
fields = ADVANCED_FIELDS

results = google_places.search_location_radius(
    latitude, longitude, radius_miles, prompt, ADVANCED_FIELDS
)
filename = f"{latitude}_{longitude}_{radius_miles}_location_radius_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
radius_miles = 1
location = "San Francisco"
prompt = f"All bakeries within {radius_miles} miles of the center of {location}"
fields = ["*"]

results = google_places.text_search(prompt, fields)
filename = f"{location}_{radius_miles}_text_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
radius_miles = 1
location = "San Francisco"
prompt = f"All bakeries within {radius_miles} miles of the center of {location}"
fields = ADVANCED_FIELDS

results = google_places.text_search(prompt, fields)
filename = f"{location}_{radius_miles}_text_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
prompt = "All restaurants"
fields = ADVANCED_FIELDS
data = {
    "locationBias": {
        "circle": {"center": {"latitude": 45.54, "longitude": -122.98}, "radius": 10.0}
    }
}
results = google_places.text_search(prompt, fields, data)
filename = f"{latitude}_{longitude}_{radius_miles}_location_radius_search.json"
save_json(filename, prompt, fields, results)

In [ ]:
prompts = DEFAULT_PROMPTS
fields = ADVANCED_FIELDS
radius_miles = 0.5
# radius_meters = radius_miles * METERS_PER_MILE
radius_meters = 200.0
location = "New York"

# tgtg_restaurant = "Chun Yang Tea" no
# line_from_csv = "TWO BRIDGES LUNCHEONETTE,-73.9911526,40.7143903" no
line_from_csv = "La Colombe Coffee Roasters,-74.0084982,40.7054378"
# line_from_csv = "Benares,-74.0091528,40.7142392"
# line_from_csv = "Blue Bottle Coffee,-74.0022231,40.7183531"
tgtg_restaurant, lon, lat = line_from_csv.split(",")
tgtg_latlon = {"latitude": float(lat), "longitude": float(lon)}

latitude = tgtg_latlon["latitude"]
longitude = tgtg_latlon["longitude"]

rect_viewpoint = get_viewport(latitude, longitude, radius_meters)

data = {"locationRestriction": {"rectangle": rect_viewpoint}}
locations = {p: list() for p in prompts}
results = {"places": []}
for prompt in prompts:
    result = google_places.text_search(prompt, fields, data)
    if "places" not in result:
        print(f"No results found for {prompt}")
    else:
        print(f"Number of results {prompt}: {len(result['places'])}")
        places = result["places"]
        names = [x['displayName']['text'].lower().lower() for x in places]
        results["places"].extend(places)
        locations[prompt].extend(names)

filename = f"{tgtg_restaurant}_{latitude}_{longitude}_{radius_miles}_location_rect_viewpoint_search.json"
save_json(filename, prompt, fields, results)
duplicate_locations = set(locations[prompts[0]]) & set(locations[prompts[1]])
found_locations = []
for places in locations.values():
    found_locations.extend(places)
print_locations = "\n".join(found_locations)
print(f"Prompts: {prompts}")
print(f"Duplicates: {len(duplicate_locations)}")
print(f"{len(found_locations)} locations found: {print_locations}")
print(f"{tgtg_restaurant} found: {tgtg_restaurant.lower() in found_locations}")